# 𝔻𝕖𝕔𝕖𝕟𝕥ℕ𝕖𝕥 𝕄𝕠𝕕𝕖𝕝 𝕥𝕖𝕤𝕥

In [1]:
import sys 
sys.path.insert(0, r"../helper")

import os

import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger



C:\Users\Prinzessin\anaconda3\envs\feta\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


## DecentNet
Basic init of a decentnet

In [2]:
from model.decentnet import *

In [3]:
model_kwargs = {
    'in_channels' : 1, # not in use yet
    'n_classes': None, # filled in the dataset
    'out_dim' :  [1, 4, 4, 8], # [1, 8, 16, 32], #[1, 16, 24, 32] # entry, decent1, decent2, decent3
    'grid_size' : 18*18,
    'criterion': torch.nn.CrossEntropyLoss(),# torch.nn.BCEWithLogitsLoss(),
    'optimizer': "sgd", # sgd adamw
    'base_lr': 0.001,
    'min_lr' : 0.00001,
    'momentum' : 0.9,
    'lr_update' : 100,
    # decentnet
    'cc_weight': 10,
    'cc_metric' : 'l2', # connection cost metric (for loss) - distance metric
    'ci_metric' : 'l2', # channel importance metric (for pruning)
    'cm_metric' : 'not implemented yet', # 'count', # crossing minimisation 
    'update_every_nth_epoch' : 1, # 5
    'pretrain_epochs' : 1, # 20
    'prune_keep' : 0.7, # 0.97, # in each epoch
    'prune_keep_total' : 0.4, # this number is not exact, depends on the prune_keep value
}

train_kwargs = {
    'result_path': "example_results", # "example_results/lightning_logs", # not in use??
    'exp_name': "experiment_name", # must include oct or retina
    'load_ckpt_file' : 'version_18/checkpoints/epoch=4-unpruned=192-val_f1=0.12.ckpt', # "version_0/checkpoints/epoch=94-unpruned=1600-val_f1=0.67.ckpt", # 'version_94/checkpoints/epoch=26-step=1080.ckpt', # change this for loading a file and using "test", if you want training, keep None
    'epochs': 3, # including the pretrain epochs - no adding up
    'img_size' : 28, #168, # keep mnist at original size, training didn't work when i increased the size ... # MNIST/MedMNIST 28 × 28 Pixel
    'batch_size': 2, # 128, # the higher the batch_size the faster the training - every iteration adds A LOT OF comp cost
    'log_every_n_steps' : 4, # lightning default: 50 # needs to be bigger than the amount of steps in an epoch (based on trainset size and batchsize)
    'device': "cuda",
    'num_workers' : 0, # 18, # 18 for computer, 0 for laptop
    'train_size' : (2 * 4), # total or percentage
    'val_size' : (2 * 4), # total or percentage
    'test_size' : 8, # total or percentage - 0 for all
}

logger = CSVLogger(os.path.join(train_kwargs["result_path"], 'lightning_logs'), name=train_kwargs["exp_name"])
log_dir = logger.log_dir

# should be filled by dataset
model_kwargs['n_classes'] = 5

Missing logger folder: example_results\lightning_logs\experiment_name


In [4]:
model = DecentNet(model_kwargs=model_kwargs, log_dir=log_dir).to("cuda")

DECENT INFO: dimensions are entry, decent1, decent2, decent3, decent1x1 == out [1, 4, 4, 8, 5]
[1.]
[7.]


In [5]:
model

DecentNet(
  (decent1): DecentLayer(
    (filter_list): ModuleList(
      (0): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_this=Parameter containing:
      tensor([0.], device='cuda:0'), n_this=Parameter containing:
      tensor([11.], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (1): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_this=Parameter containing:
      tensor([9.], device='cuda:0'), n_this=Parameter containing:
      tensor([11.], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (2): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_this=Parameter containing:
      tensor([2.], device='cuda:0'), n_this=Parameter containing:
      tensor([8.], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (3): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_this=Parameter containing:
      tensor([16.], device='cuda:0'), n_this=Parameter containing:
      tensor([4.], device='cuda:0'